## Fermihedral: On the Optimal Compilation for Fermion-to-Qubit Encoding

Please make sure your are using the virtual environment, and `kissat` is built. If not, run `python3 prepare.py` to prepare the environment and packages.

In [2]:
%matplotlib inline

from math import log2
import matplotlib.pyplot as plt
from scipy.stats import linregress
import numpy as np

plt.style.use('classic')
plt.rc("font", size=28, family="serif")

#### Experiment: Average Pauli weight per-Majorana operator (small scale)

In [ ]:
from fermihedral import DescentSolver
from fermihedral.satutil import Kissat
from fermihedral.majorana import get_bk_weight

nmodes = []
avg_fh_weight = []
avg_bk_weight = []

for i in range(1, 9):
    solver = DescentSolver(i, independence=True, vacuum=False)
    _, weight = solver.solve(progress=True, solver_init=Kissat, solver_args=[10 * 60])
    nmodes.append(i)
    avg_fh_weight.append(weight / (2 * i))
    avg_bk_weight.append(get_bk_weight(i) / (2 * i))


In [ ]:
# plot result

def log_regress(x, y):
    log_x = [log2(i) for i in x]
    a, b, _, _, _ = linregress(log_x, y)
    return a, b, [a * i + b for i in log_x]

plt.clf()
plt.figure(figsize=(7, 6))
plt.subplots_adjust(left=0.12, right=0.98, top=0.95, bottom=0.15)
plt.plot(nmodes[:8], avg_bk_weight[:8], label="Bravyi-Kitaev", marker="o", markerfacecolor='none', markeredgecolor='brown', color='brown', markersize=12, linewidth=2, markeredgewidth=2)
plt.plot(nmodes[:8], avg_fh_weight[:8], label="Full SAT", marker="x", markerfacecolor='none', markeredgecolor='blue', color='blue', markersize=12, linewidth=2, markeredgewidth=2)
bk_a, bk_b, bk_data = log_regress(nmodes[:8], avg_bk_weight[:8])
plt.plot(nmodes[:8], bk_data, label=f"{bk_a:.2f}log$_2$(N)+{bk_b:.2f}", color="brown", ls="--", linewidth=1)
fh_a, fh_b, fh_data = log_regress(nmodes[:8], avg_fh_weight[:8])
plt.plot(nmodes[:8], fh_data, label=f"{fh_a:.2f}log$_2$(N)+{fh_b:.2f}", color="blue", ls="--", linewidth=1)
plt.legend(loc='lower right', bbox_to_anchor=(1, 0), ncol=1, fontsize=21)
plt.grid()
plt.ylim(0.5, 3.7)
plt.xlim(0.5, 8.5)
plt.yticks([1, 2, 3])
plt.xlabel("Modes/n")
plt.ylabel("Pauli Weight/n")
plt.show()

#### Experiment: Average Pauli weight per-Majorana operator (larger scale)

Note: this step may take extremely long time. If you want to go for the noisy simulation, then you can bypass this step and the following experiments.

In [ ]:
from fermihedral import DescentSolver
from fermihedral.satutil import Kissat
from fermihedral.majorana import get_bk_weight

nmodes = []
fh_weight = []
avg_fh_weight = []
avg_bk_weight = []

TIMEOUT = 72 * 60 * 60 # 3 Days, adjust this number based on your computer's performance

for i in range(1, 9):
    solver = DescentSolver(i, independence=False, vacuum=False)
    _, weight = solver.solve(progress=True, solver_init=Kissat, solver_args=[TIMEOUT])
    nmodes.append(i)
    fh_weight.append(weight)
    avg_fh_weight.append(weight / (2 * i))
    avg_bk_weight.append(get_bk_weight(i) / (2 * i))

In [ ]:
# plot

exp_n_modes = nmodes[8:]
exp_avg_bk_weight = avg_bk_weight[8:]
exp_avg_fermihedral_weight = avg_fh_weight[8:]

plt.clf()
plt.figure(figsize=(14, 6))
plt.subplots_adjust(left=0.07, right=0.98, top=0.95, bottom=0.15, wspace=0.27)

plt.subplot(1, 2, 1)
plt.plot(exp_n_modes, exp_avg_bk_weight, label="Bravyi-Kitaev",
         marker="o", markerfacecolor='none', markeredgecolor='brown', color='brown', markersize=12, markeredgewidth=2, linewidth=2)
plt.plot(exp_n_modes, exp_avg_fermihedral_weight,
         label="SAT w/o Alg.", marker="x", markerfacecolor='none', markeredgecolor='blue', color='blue', markersize=12, markeredgewidth=2, linewidth=2)
bk_a, bk_b, bk_data = log_regress(exp_n_modes, exp_avg_bk_weight)
plt.plot(exp_n_modes, bk_data, color="brown", ls="--", linewidth=1)
fh_a, fh_b, fh_data = log_regress(exp_n_modes, exp_avg_fermihedral_weight)
plt.plot(exp_n_modes, fh_data, color="blue", ls="--", linewidth=1)
plt.legend(loc="upper left", fontsize=23)
plt.grid()
plt.xlabel("Modes/n")
plt.ylabel("Pauli Weight/n")
plt.yticks([3.0, 4.0, 5.0])


plt.subplot(1, 2, 2)
improvement = [(exp_avg_bk_weight[i] - exp_avg_fermihedral_weight[i])
               * 100 / exp_avg_bk_weight[i] for i in range(len(exp_n_modes))]
plt.bar(exp_n_modes, improvement, color="white",
        edgecolor="black", linewidth=1.5)
plt.plot([8] + exp_n_modes + [20], [avg_improvement for _ in range(
    len(exp_n_modes) + 2)], color='brown', linewidth=3)
plt.grid()
plt.xlabel("Modes/n")
plt.ylabel("Improvement/%")
plt.yticks([5, 15, 25])
plt.show()

#### Experiment: Probability of $n$ $A_k$'s holds simultaneously

Note: this step dependes on the previous step. Since the previous takes really long, you could skip this if you only want the noisy simulation part.

In [3]:
from fermihedral import MajoranaModel
from fermihedral.pauli import check_algebraic_independent
from fermihedral.satutil import Kissat

